In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
shear_range = 0.1,
zoom_range = 0.1,
horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [4]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Dataset/Car damage/body/training',
target_size = (224, 224),
batch_size = 10,
class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Dataset/Car damage/body/training',
target_size = (224, 224),
batch_size = 10,
class_mode = 'categorical')

Found 979 images belonging to 3 classes.
Found 979 images belonging to 3 classes.


In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [6]:
IMAGE_SIZE = [224, 224]
train_path = '/content/drive/MyDrive/Dataset/Car damage/body/training'
valid_path = '/content/drive/MyDrive/Dataset/Car damage/body/validation'

In [7]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


In [8]:
for layer in vgg16.layers:
  layer.trainable = False

In [9]:
folders = glob('/content/drive/MyDrive/Dataset/Car damage/body/training/*')

In [10]:
folders

['/content/drive/MyDrive/Dataset/Car damage/body/training/02-side',
 '/content/drive/MyDrive/Dataset/Car damage/body/training/01-rear',
 '/content/drive/MyDrive/Dataset/Car damage/body/training/00-front']

In [11]:
x = Flatten() (vgg16.output)

In [12]:
len(folders)

3

In [13]:
prediction = Dense(len(folders), activation='softmax')(x)


In [14]:
model = Model(inputs=vgg16.input, outputs=prediction)

In [15]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [16]:
model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)


In [17]:
r = model.fit_generator(
training_set,
validation_data=test_set,
epochs=25,
steps_per_epoch=len(training_set),
validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/25
98/98 [==============================] - 1025s 10s/step - loss: 1.3726 - accuracy: 0.5087 - val_loss: 0.6431 - val_accuracy: 0.7671
Epoch 2/25
98/98 [==============================] - 1022s 10s/step - loss: 0.7143 - accuracy: 0.7201 - val_loss: 0.4365 - val_accuracy: 0.8243
Epoch 3/25
98/98 [==============================] - 1024s 10s/step - loss: 0.4599 - accuracy: 0.8304 - val_loss: 0.3137 - val_accuracy: 0.8836
Epoch 4/25
98/98 [==============================] - 1021s 10s/step - loss: 0.3622 - accuracy: 0.8631 - val_loss: 0.2275 - val_accuracy: 0.9142
Epoch 5/25
98/98 [==============================] - 1029s 11s/step - loss: 0.2830 - accuracy: 0.8825 - val_loss: 0.2081 - val_accuracy: 0.9254
Epoch 6/25
98/98 [==============================] - 1025s 11s/step - loss: 0.2023 - accuracy: 0.9305 - val_loss: 0.1584 - val_accuracy: 0.9489
Epoch 7/25
98/98 [==============================] - 1026s 11s/step - loss: 0.1646 - accuracy: 0.9510 - val_loss: 0.1743 - val_accuracy: 0.9438

In [18]:
from tensorflow.keras.models import load_model
model.save('/content/drive/MyDrive/Dataset/Car damage/body')

In [19]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [21]:
model = load_model('/content/drive/MyDrive/Dataset/Car damage/body')


In [26]:
def detect(frame):
  img = cv2.resize(frame,(224,224))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

  if(np.max(img)>1):
    img = img/255.0
  img = np.array([img])
  prediction = model.predict(img)
  label = ["front","rear","side"]
  preds = label[np.argmax(prediction)]
  return preds



In [27]:
import numpy as np


In [28]:
data = "/content/drive/MyDrive/Dataset/Car damage/body/training/00-front/0003.JPEG"
image = cv2.imread(data)
print(detect(image))

1/1 [==============================] - 1s 769ms/step
front
